# Collaborative Filtering implementation in Keras

To demonstrate the use of modern Machine learning libraries with Automatic Differenciation we re-implement 
the Collaborative Filtering example using `keras` library.

## Preliminaries

### Imports

In [95]:
import os
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,KFold

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
print(tf.__version__)

%matplotlib inline

1.11.0


### Tensorflow Config

In [2]:
#
from tensorflow.keras.backend import set_session

config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)

sess = tf.Session(config=config)

set_session(sess)  # set this TensorFlow session as the default session for Keras

### Random Seed

In [3]:
seed=5543
np.random.seed(seed)

### Data

In [4]:
raw_dir="../raw/ml-1m"
data_dir="../data/MovieLens"

In [5]:
if not (os.path.exists(data_dir)):
    os.mkdir(data_dir)

In [6]:
filename=raw_dir+"/ratings.dat"

In [7]:
data_all=pd.read_csv(filename,sep="::",header=None,names=["userId","movieId","rating","TimeStamp"])

/home/manel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [8]:
data_all.head()

,userId,movieId,rating,TimeStamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### Train Test Split

In this example we do not handle carefuly new users or new movies (we just asign them at random).

A production product would treat new users/movies separately.

In [9]:
userEncoder=LabelEncoder()
movieEncoder=LabelEncoder()

In [10]:
users_all=userEncoder.fit_transform(data_all[["userId"]].values.ravel())
movies_all=movieEncoder.fit_transform(data_all[["movieId"]].values.ravel())
ratings_all=data_all[["rating"]].values.ravel()

In [11]:
users,users_test,movies,movies_test,ratings, ratings_test=train_test_split(users_all,movies_all,ratings_all,test_size=0.15)
users_train,users_val, movies_train,movies_val, ratings_train,ratings_val=train_test_split(users,movies,ratings,test_size=0.15)
users_train.shape,users_val.shape,users_test.shape

((722650,), (127527,), (150032,))

In [12]:
unkown_users=~np.isin(users_val,users)
unknown_movies=~np.isin(movies_val,movies)
print(unkown_users.sum(),unknown_movies.sum())

0 0


In [13]:
unique_users=userEncoder.classes_
unique_movies=movieEncoder.classes_

In [14]:
N_users=len(unique_users)
N_movies=len(unique_movies)
print(N_users,N_movies)

6040 3706


In [15]:
mu=ratings_train.mean()
drating=np.mean((ratings_train-mu)**2)
print(mu,drating)

3.5812398809935653 1.2487443009293984


In [40]:
dratings_train=ratings_train-mu
dratings_val=ratings_val-mu
dratings_test=ratings_test-mu

## Keras Factorization Model

We define a metric for reports (keras by default will include the penalty on the metrics report)

In [67]:
def mse_err(y_true, y_pred):
    #return K.mean((y_pred-y_true)**2,axis=-1)
    return K.mean(K.square(y_pred - y_true), axis=-1)

We implement a call back to that we can see optimization progress every 10 epochs or so.

In [68]:
class ReportCallback(keras.callbacks.Callback):
    def __init__(self,frequency,use_val=False):
        self.freq=frequency
        self.use_val=use_val
        self.separator=" || "
        if not(self.use_val):
            self.separator="\n"
    def on_epoch_end(self, epoch, logs={}):
        if (epoch % self.freq ==0):
            train_loss=logs["loss"]
            train_error=logs["mse_err"]
            print(f"\t{epoch}: TRAIN Loss {train_loss:.4f},  Err {train_error:.4f}",end=self.separator)
            if self.use_val:
                val_loss=logs["val_loss"]
                val_error=logs["val_mse_err"]
                print(f"VAL Loss {val_loss:.4f}, Err {val_error:.4f}")

### Keras Rating Model

This implements
\begin{equation}
	\Delta \hat{r}_{i,u} =b_u + b_i + p_u^T q_i
\end{equation}

Where 
$$
    \Delta \hat{r}_{i,u} = \hat{r}_{i,u} - \mu
$$
using `keras` layers.

In [69]:
def collaborative_filter(U,M,F,penalty):
    users = keras.layers.Input(shape=(1,))
    movies = keras.layers.Input(shape=(1,))
    bu= keras.layers.Embedding(U,1,input_length=1,
                              embeddings_initializer=keras.initializers.RandomNormal(stddev=0.0001),
                              )(users)
    bm= keras.layers.Embedding(M,1,input_length=1,
                              embeddings_initializer=keras.initializers.RandomNormal(stddev=0.0001),
                              )(movies)
    pu= keras.layers.Embedding(U,F,input_length=1,
                               embeddings_initializer=keras.initializers.RandomNormal(stddev=1/np.sqrt(F)),
                               embeddings_regularizer=keras.regularizers.l2(penalty))(users)
    pm= keras.layers.Embedding(M,F,input_length=1,
                               embeddings_initializer=keras.initializers.RandomNormal(stddev=1/np.sqrt(F)),
                               embeddings_regularizer=keras.regularizers.l2(penalty))(movies)
    interaction=keras.layers.Dot(axes=-1)([pu,pm])
    agg=keras.layers.Add()([bu,bm,interaction])
    r_hat=keras.layers.Flatten()(agg)
    model = keras.models.Model(inputs=[users,movies], outputs=r_hat)
    return model

In [70]:
penalty=1e-5
F=10

We  can create a model for our choice of embedding dimention and penalty

In [71]:
model=collaborative_filter(N_users,N_movies,F,penalty)

We can compile the model and use it like any other `keras` model.

In [72]:
model.compile(optimizer=keras.optimizers.Adam(),
              metrics=[mse_err],
              loss='mean_squared_error')

In [73]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_33 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_34 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_66 (Embedding)        (None, 1, 10)        60400       input_33[0][0]                   
__________________________________________________________________________________________________
embedding_67 (Embedding)        (None, 1, 10)        37060       input_34[0][0]                   
__________________________________________________________________________________________________
embedding_

### Model Tranining

We can train as before, but now we must pass multiple inputs as a list of python objects.

In [74]:

model.fit([users_train,movies_train],dratings_train,
          epochs=200,batch_size=5000,
          callbacks=[ReportCallback(1,True)],
          verbose=0,
          validation_data=([users_val,movies_val],dratings_val))

/home/manel/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


	0: TRAIN Loss 1.3730,  Err 1.2827 || VAL Loss 1.3048, Err 1.2209
	1: TRAIN Loss 1.2413,  Err 1.1624 || VAL Loss 1.2022, Err 1.1282
	2: TRAIN Loss 1.1442,  Err 1.0744 || VAL Loss 1.1256, Err 1.0600
	3: TRAIN Loss 1.0713,  Err 1.0092 || VAL Loss 1.0680, Err 1.0093
	4: TRAIN Loss 1.0161,  Err 0.9604 || VAL Loss 1.0243, Err 0.9715
	5: TRAIN Loss 0.9737,  Err 0.9235 || VAL Loss 0.9907, Err 0.9429
	6: TRAIN Loss 0.9408,  Err 0.8951 || VAL Loss 0.9646, Err 0.9211
	7: TRAIN Loss 0.9147,  Err 0.8730 || VAL Loss 0.9441, Err 0.9041
	8: TRAIN Loss 0.8938,  Err 0.8554 || VAL Loss 0.9278, Err 0.8909
	9: TRAIN Loss 0.8769,  Err 0.8412 || VAL Loss 0.9146, Err 0.8803
	10: TRAIN Loss 0.8628,  Err 0.8295 || VAL Loss 0.9039, Err 0.8718
	11: TRAIN Loss 0.8509,  Err 0.8196 || VAL Loss 0.8951, Err 0.8647
	12: TRAIN Loss 0.8408,  Err 0.8111 || VAL Loss 0.8878, Err 0.8588
	13: TRAIN Loss 0.8319,  Err 0.8035 || VAL Loss 0.8816, Err 0.8537
	14: TRAIN Loss 0.8240,  Err 0.7965 || VAL Loss 0.8762, Err 0.8492
	15: 

	123: TRAIN Loss 0.6626,  Err 0.5957 || VAL Loss 0.8099, Err 0.7430
	124: TRAIN Loss 0.6625,  Err 0.5955 || VAL Loss 0.8098, Err 0.7428
	125: TRAIN Loss 0.6623,  Err 0.5953 || VAL Loss 0.8097, Err 0.7426
	126: TRAIN Loss 0.6622,  Err 0.5951 || VAL Loss 0.8097, Err 0.7425
	127: TRAIN Loss 0.6621,  Err 0.5949 || VAL Loss 0.8096, Err 0.7424
	128: TRAIN Loss 0.6620,  Err 0.5947 || VAL Loss 0.8095, Err 0.7423
	129: TRAIN Loss 0.6618,  Err 0.5945 || VAL Loss 0.8094, Err 0.7421
	130: TRAIN Loss 0.6617,  Err 0.5944 || VAL Loss 0.8093, Err 0.7419
	131: TRAIN Loss 0.6616,  Err 0.5942 || VAL Loss 0.8092, Err 0.7418
	132: TRAIN Loss 0.6615,  Err 0.5940 || VAL Loss 0.8091, Err 0.7416
	133: TRAIN Loss 0.6614,  Err 0.5938 || VAL Loss 0.8090, Err 0.7415
	134: TRAIN Loss 0.6613,  Err 0.5937 || VAL Loss 0.8089, Err 0.7413
	135: TRAIN Loss 0.6612,  Err 0.5935 || VAL Loss 0.8088, Err 0.7412
	136: TRAIN Loss 0.6610,  Err 0.5934 || VAL Loss 0.8088, Err 0.7411
	137: TRAIN Loss 0.6610,  Err 0.5932 || VAL Loss

KeyboardInterrupt: 

In [75]:
Y_pred=model.predict([users_val,movies_val]).ravel()
loss=np.mean((dratings_val-Y_pred)**2)
print(loss)

0.7354717149159153


## Hyperparameter search

In [79]:
results=[]
best_loss=1e10
best_F=None
best_penalty=None
steps=100
if True:
  for F in [1,5,10,15,20,30,50,100,150]: #[0,1,5,10,20,30,50,75,100,125,150]:
    for penalty in [0,1e-6,5e-6,1e-5,2e-5,3e-5,5e-5,1e-4]:
        print()
        print(f"F {F}, penalty {penalty} :")
        penalty=penalty
        model=collaborative_filter(N_users,N_movies,F,penalty)
        model.compile(optimizer=keras.optimizers.Adam(),
              metrics=[mse_err],
              loss='mean_squared_error')
        model.fit([users_train,movies_train],dratings_train,
          epochs=steps,batch_size=5000,
          callbacks=[ReportCallback(1,True)],
          verbose=0,
          validation_data=([users_val,movies_val],dratings_val))
        Y_pred=mu+model.predict([users_val,movies_val]).ravel()
        loss=np.mean((ratings_val-Y_pred)**2)
        results.append((F,penalty,loss))
        if loss<best_loss:
            best_loss=loss
            best_F=F
            best_penalty=penalty
        print()
        print(f"==> {F},{penalty},{loss} == best ({best_F},{best_penalty},{best_loss}) =============")


F 1, penalty 0 :


/home/manel/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


	0: TRAIN Loss 2.0850,  Err 2.0850 || VAL Loss 1.9501, Err 1.9501
	1: TRAIN Loss 1.8104,  Err 1.8104 || VAL Loss 1.7180, Err 1.7180
	2: TRAIN Loss 1.6053,  Err 1.6053 || VAL Loss 1.5429, Err 1.5429
	3: TRAIN Loss 1.4498,  Err 1.4498 || VAL Loss 1.4088, Err 1.4088
	4: TRAIN Loss 1.3303,  Err 1.3303 || VAL Loss 1.3053, Err 1.3053
	5: TRAIN Loss 1.2372,  Err 1.2372 || VAL Loss 1.2239, Err 1.2239
	6: TRAIN Loss 1.1638,  Err 1.1638 || VAL Loss 1.1595, Err 1.1595
	7: TRAIN Loss 1.1054,  Err 1.1054 || VAL Loss 1.1077, Err 1.1077
	8: TRAIN Loss 1.0583,  Err 1.0583 || VAL Loss 1.0658, Err 1.0658
	9: TRAIN Loss 1.0199,  Err 1.0199 || VAL Loss 1.0314, Err 1.0314
	10: TRAIN Loss 0.9883,  Err 0.9883 || VAL Loss 1.0031, Err 1.0031
	11: TRAIN Loss 0.9622,  Err 0.9622 || VAL Loss 0.9795, Err 0.9795
	12: TRAIN Loss 0.9404,  Err 0.9404 || VAL Loss 0.9597, Err 0.9597
	13: TRAIN Loss 0.9220,  Err 0.9220 || VAL Loss 0.9429, Err 0.9429
	14: TRAIN Loss 0.9064,  Err 0.9064 || VAL Loss 0.9287, Err 0.9287
	15: 

	22: TRAIN Loss 0.8422,  Err 0.8386 || VAL Loss 0.8644, Err 0.8608
	23: TRAIN Loss 0.8379,  Err 0.8344 || VAL Loss 0.8606, Err 0.8572
	24: TRAIN Loss 0.8342,  Err 0.8308 || VAL Loss 0.8573, Err 0.8539
	25: TRAIN Loss 0.8308,  Err 0.8275 || VAL Loss 0.8544, Err 0.8511
	26: TRAIN Loss 0.8279,  Err 0.8246 || VAL Loss 0.8518, Err 0.8486
	27: TRAIN Loss 0.8252,  Err 0.8220 || VAL Loss 0.8496, Err 0.8464
	28: TRAIN Loss 0.8229,  Err 0.8197 || VAL Loss 0.8475, Err 0.8444
	29: TRAIN Loss 0.8207,  Err 0.8177 || VAL Loss 0.8457, Err 0.8427
	30: TRAIN Loss 0.8188,  Err 0.8159 || VAL Loss 0.8441, Err 0.8411
	31: TRAIN Loss 0.8172,  Err 0.8142 || VAL Loss 0.8427, Err 0.8398
	32: TRAIN Loss 0.8156,  Err 0.8127 || VAL Loss 0.8414, Err 0.8386
	33: TRAIN Loss 0.8142,  Err 0.8114 || VAL Loss 0.8403, Err 0.8375
	34: TRAIN Loss 0.8130,  Err 0.8102 || VAL Loss 0.8393, Err 0.8365
	35: TRAIN Loss 0.8118,  Err 0.8091 || VAL Loss 0.8384, Err 0.8357
	36: TRAIN Loss 0.8107,  Err 0.8081 || VAL Loss 0.8375, Err 0.

	43: TRAIN Loss 0.8098,  Err 0.8028 || VAL Loss 0.8359, Err 0.8290
	44: TRAIN Loss 0.8090,  Err 0.8022 || VAL Loss 0.8353, Err 0.8287
	45: TRAIN Loss 0.8082,  Err 0.8017 || VAL Loss 0.8347, Err 0.8283
	46: TRAIN Loss 0.8075,  Err 0.8012 || VAL Loss 0.8341, Err 0.8279
	47: TRAIN Loss 0.8067,  Err 0.8006 || VAL Loss 0.8336, Err 0.8276
	48: TRAIN Loss 0.8060,  Err 0.8001 || VAL Loss 0.8331, Err 0.8273
	49: TRAIN Loss 0.8053,  Err 0.7996 || VAL Loss 0.8326, Err 0.8269
	50: TRAIN Loss 0.8046,  Err 0.7990 || VAL Loss 0.8321, Err 0.8266
	51: TRAIN Loss 0.8039,  Err 0.7985 || VAL Loss 0.8316, Err 0.8262
	52: TRAIN Loss 0.8032,  Err 0.7979 || VAL Loss 0.8310, Err 0.8258
	53: TRAIN Loss 0.8025,  Err 0.7972 || VAL Loss 0.8305, Err 0.8253
	54: TRAIN Loss 0.8017,  Err 0.7965 || VAL Loss 0.8299, Err 0.8248
	55: TRAIN Loss 0.8009,  Err 0.7958 || VAL Loss 0.8293, Err 0.8243
	56: TRAIN Loss 0.8000,  Err 0.7950 || VAL Loss 0.8286, Err 0.8237
	57: TRAIN Loss 0.7991,  Err 0.7942 || VAL Loss 0.8278, Err 0.

...

	45: TRAIN Loss 0.2791,  Err 0.0939 || VAL Loss 1.1572, Err 0.9717
	46: TRAIN Loss 0.2780,  Err 0.0925 || VAL Loss 1.1603, Err 0.9745
	47: TRAIN Loss 0.2770,  Err 0.0912 || VAL Loss 1.1631, Err 0.9771
	48: TRAIN Loss 0.2760,  Err 0.0900 || VAL Loss 1.1660, Err 0.9798
	49: TRAIN Loss 0.2750,  Err 0.0889 || VAL Loss 1.1686, Err 0.9822
	50: TRAIN Loss 0.2741,  Err 0.0878 || VAL Loss 1.1712, Err 0.9847
	51: TRAIN Loss 0.2732,  Err 0.0867 || VAL Loss 1.1737, Err 0.9869
	52: TRAIN Loss 0.2724,  Err 0.0857 || VAL Loss 1.1762, Err 0.9893
	53: TRAIN Loss 0.2716,  Err 0.0848 || VAL Loss 1.1785, Err 0.9915
	54: TRAIN Loss 0.2708,  Err 0.0839 || VAL Loss 1.1809, Err 0.9937
	55: TRAIN Loss 0.2700,  Err 0.0830 || VAL Loss 1.1832, Err 0.9959
	56: TRAIN Loss 0.2693,  Err 0.0821 || VAL Loss 1.1854, Err 0.9980
	57: TRAIN Loss 0.2686,  Err 0.0813 || VAL Loss 1.1875, Err 1.0000
	58: TRAIN Loss 0.2680,  Err 0.0806 || VAL Loss 1.1896, Err 1.0020
	59: TRAIN Loss 0.2674,  Err 0.0799 || VAL Loss 1.1916, Err 1.

	66: TRAIN Loss 0.4180,  Err 0.1495 || VAL Loss 1.1393, Err 0.8706
	67: TRAIN Loss 0.4176,  Err 0.1490 || VAL Loss 1.1400, Err 0.8712
	68: TRAIN Loss 0.4173,  Err 0.1485 || VAL Loss 1.1406, Err 0.8717
	69: TRAIN Loss 0.4170,  Err 0.1481 || VAL Loss 1.1414, Err 0.8723
	70: TRAIN Loss 0.4167,  Err 0.1476 || VAL Loss 1.1422, Err 0.8729
	71: TRAIN Loss 0.4164,  Err 0.1471 || VAL Loss 1.1428, Err 0.8734
	72: TRAIN Loss 0.4161,  Err 0.1468 || VAL Loss 1.1434, Err 0.8739
	73: TRAIN Loss 0.4158,  Err 0.1463 || VAL Loss 1.1441, Err 0.8745
	74: TRAIN Loss 0.4156,  Err 0.1459 || VAL Loss 1.1449, Err 0.8751
	75: TRAIN Loss 0.4153,  Err 0.1456 || VAL Loss 1.1454, Err 0.8755
	76: TRAIN Loss 0.4151,  Err 0.1452 || VAL Loss 1.1460, Err 0.8760
	77: TRAIN Loss 0.4148,  Err 0.1448 || VAL Loss 1.1465, Err 0.8764
	78: TRAIN Loss 0.4146,  Err 0.1445 || VAL Loss 1.1470, Err 0.8768
	79: TRAIN Loss 0.4143,  Err 0.1441 || VAL Loss 1.1477, Err 0.8774
	80: TRAIN Loss 0.4141,  Err 0.1439 || VAL Loss 1.1483, Err 0.

	87: TRAIN Loss 0.6132,  Err 0.3174 || VAL Loss 1.0534, Err 0.7577
	88: TRAIN Loss 0.6131,  Err 0.3171 || VAL Loss 1.0535, Err 0.7577
	89: TRAIN Loss 0.6130,  Err 0.3169 || VAL Loss 1.0537, Err 0.7578
	90: TRAIN Loss 0.6129,  Err 0.3168 || VAL Loss 1.0540, Err 0.7579
	91: TRAIN Loss 0.6129,  Err 0.3165 || VAL Loss 1.0543, Err 0.7580
	92: TRAIN Loss 0.6128,  Err 0.3163 || VAL Loss 1.0544, Err 0.7581
	93: TRAIN Loss 0.6127,  Err 0.3161 || VAL Loss 1.0546, Err 0.7581
	94: TRAIN Loss 0.6127,  Err 0.3159 || VAL Loss 1.0547, Err 0.7581
	95: TRAIN Loss 0.6126,  Err 0.3158 || VAL Loss 1.0549, Err 0.7582
	96: TRAIN Loss 0.6125,  Err 0.3156 || VAL Loss 1.0550, Err 0.7583
	97: TRAIN Loss 0.6125,  Err 0.3154 || VAL Loss 1.0551, Err 0.7582
	98: TRAIN Loss 0.6125,  Err 0.3153 || VAL Loss 1.0554, Err 0.7584
	99: TRAIN Loss 0.6124,  Err 0.3151 || VAL Loss 1.0555, Err 0.7584

==> 150,2e-05,0.7584373678137889 == best (10,2e-05,0.7366513533519797) =============

F 150, penalty 3e-05 :
	0: TRAIN Loss 1.39

	7: TRAIN Loss 0.8719,  Err 0.8028 || VAL Loss 0.8888, Err 0.8208
	8: TRAIN Loss 0.8616,  Err 0.7931 || VAL Loss 0.8801, Err 0.8131
	9: TRAIN Loss 0.8526,  Err 0.7851 || VAL Loss 0.8729, Err 0.8068
	10: TRAIN Loss 0.8447,  Err 0.7779 || VAL Loss 0.8667, Err 0.8012
	11: TRAIN Loss 0.8378,  Err 0.7717 || VAL Loss 0.8613, Err 0.7963
	12: TRAIN Loss 0.8319,  Err 0.7663 || VAL Loss 0.8568, Err 0.7923
	13: TRAIN Loss 0.8268,  Err 0.7615 || VAL Loss 0.8529, Err 0.7888
	14: TRAIN Loss 0.8223,  Err 0.7575 || VAL Loss 0.8495, Err 0.7858
	15: TRAIN Loss 0.8184,  Err 0.7539 || VAL Loss 0.8467, Err 0.7836
	16: TRAIN Loss 0.8151,  Err 0.7511 || VAL Loss 0.8442, Err 0.7815
	17: TRAIN Loss 0.8122,  Err 0.7486 || VAL Loss 0.8422, Err 0.7797
	18: TRAIN Loss 0.8097,  Err 0.7463 || VAL Loss 0.8405, Err 0.7782
	19: TRAIN Loss 0.8075,  Err 0.7446 || VAL Loss 0.8390, Err 0.7770
	20: TRAIN Loss 0.8056,  Err 0.7429 || VAL Loss 0.8377, Err 0.7757
	21: TRAIN Loss 0.8040,  Err 0.7413 || VAL Loss 0.8364, Err 0.774

	28: TRAIN Loss 0.8097,  Err 0.8056 || VAL Loss 0.8273, Err 0.8233
	29: TRAIN Loss 0.8093,  Err 0.8052 || VAL Loss 0.8269, Err 0.8230
	30: TRAIN Loss 0.8089,  Err 0.8049 || VAL Loss 0.8266, Err 0.8228
	31: TRAIN Loss 0.8085,  Err 0.8046 || VAL Loss 0.8264, Err 0.8226
	32: TRAIN Loss 0.8082,  Err 0.8043 || VAL Loss 0.8262, Err 0.8225
	33: TRAIN Loss 0.8079,  Err 0.8041 || VAL Loss 0.8260, Err 0.8223
	34: TRAIN Loss 0.8077,  Err 0.8039 || VAL Loss 0.8258, Err 0.8221
	35: TRAIN Loss 0.8074,  Err 0.8037 || VAL Loss 0.8256, Err 0.8220
	36: TRAIN Loss 0.8073,  Err 0.8035 || VAL Loss 0.8255, Err 0.8218
	37: TRAIN Loss 0.8071,  Err 0.8033 || VAL Loss 0.8254, Err 0.8218
	38: TRAIN Loss 0.8069,  Err 0.8032 || VAL Loss 0.8253, Err 0.8217
	39: TRAIN Loss 0.8067,  Err 0.8031 || VAL Loss 0.8252, Err 0.8216
	40: TRAIN Loss 0.8066,  Err 0.8030 || VAL Loss 0.8251, Err 0.8215
	41: TRAIN Loss 0.8065,  Err 0.8028 || VAL Loss 0.8250, Err 0.8215
	42: TRAIN Loss 0.8064,  Err 0.8027 || VAL Loss 0.8250, Err 0.

In [80]:
Y_pred.shape

(127527,)

In [81]:
fits=pd.DataFrame(results,columns=["F","penalty","val_rms"])
fits.head()

,F,penalty,val_rms
0,1,0.000000,0.810363
1,1,0.000001,0.785315
2,1,0.000005,0.784891
3,1,0.000010,0.783396
4,1,0.000020,0.786322


In [82]:
summary=pd.pivot_table(fits,index="F",columns="penalty",values="val_rms")
summary

penalty,0.0,1e-06,5e-06,1e-05,2e-05,3e-05,5e-05,0.0001
F,,,,,,,,
1,0.810363,0.785315,0.784891,0.783396,0.786322,0.789517,0.797876,0.820675
5,0.786196,0.774025,0.758775,0.742496,0.747487,0.755300,0.774091,0.819684
10,0.824282,0.800488,0.763996,0.745207,0.736651,0.742129,0.768487,0.820113
15,0.880004,0.841870,0.783684,0.754371,0.737541,0.740076,0.767632,0.820154
20,0.931815,0.880725,0.807047,0.765119,0.739928,0.738499,0.767092,0.820097
30,1.041985,0.964705,0.851967,0.795744,0.747960,0.738415,0.766855,0.820306
50,1.251038,1.113355,0.939751,0.840785,0.759068,0.738797,0.767313,0.820117
100,1.598904,1.332158,1.043215,0.895964,0.764677,0.737937,0.768154,0.820301
150,1.699842,1.371497,1.053142,0.884558,0.758437,0.736605,0.768550,0.820279


In [83]:
best=fits.iloc[fits["val_rms"].idxmin()]
best

F          150.000000
penalty      0.000030
val_rms      0.736605
Name: 69, dtype: float64

In [84]:
import seaborn as sns

cm = sns.light_palette("#60FF60", reverse=True, as_cmap=True)

s = summary.style.highlight_min(axis=None)
s

penalty,0.0,1e-06,5e-06,1e-05,2e-05,3e-05,5e-05,0.0001
F,,,,,,,,
1,0.810363,0.785315,0.784891,0.783396,0.786322,0.789517,0.797876,0.820675
5,0.786196,0.774025,0.758775,0.742496,0.747487,0.7553,0.774091,0.819684
10,0.824282,0.800488,0.763996,0.745207,0.736651,0.742129,0.768487,0.820113
15,0.880004,0.84187,0.783684,0.754371,0.737541,0.740076,0.767632,0.820154
20,0.931815,0.880725,0.807047,0.765119,0.739928,0.738499,0.767092,0.820097
30,1.04199,0.964705,0.851967,0.795744,0.74796,0.738415,0.766855,0.820306
50,1.25104,1.11336,0.939751,0.840785,0.759068,0.738797,0.767313,0.820117
100,1.5989,1.33216,1.04322,0.895964,0.764677,0.737937,0.768154,0.820301
150,1.69984,1.3715,1.05314,0.884558,0.758437,0.736605,0.76855,0.820279


In [85]:
F_best=int(best["F"])
penalty_best=best["penalty"]
F_best,penalty_best

(150, 3e-05)

## Cross Validation

In [94]:
nK=5
kfold=KFold(nK,shuffle=True)
folds=[]
for fold in kfold.split(users):
    folds.append(fold)

In [97]:
def ratings_cross_validate(model,users,movies,ratings,folds):
    accuracies=[]
    count=0
    for train,val in folds:
        print()
        print("============= Fold",count+1,"===========")
        users_train=users[train]
        movies_train=movies[train]
        ratings_train=ratings[train]
        mu=ratings_train.mean()
        dratings_train=ratings_train-mu
        users_val=users[val]
        movies_val=movies[val]
        ratings_val=ratings[val]
        dratings_val=ratings_val-mu
        model.fit([users_train,movies_train],dratings_train,
          epochs=steps,batch_size=5000,
          callbacks=[ReportCallback(1,True)],
          verbose=0,
          validation_data=([users_val,movies_val],dratings_val))
        Y_pred=mu+model.predict([users_val,movies_val]).ravel()
        loss=np.mean((ratings_val-Y_pred)**2)
        
        accuracies.append(loss)
        print("======= fold",count+1,"loss",loss,"============")
        print()
        count+=1
    accuracies=np.array(accuracies)
    return accuracies.mean()

In [99]:
results=[]
best_loss=1e10
best_F=None
steps=100
best_penalty=2e-5
if True:
  for F in [5,10,15]: 
        print()
        print(f"F {F} :")
        model=collaborative_filter(N_users,N_movies,F,best_penalty)
        model.compile(optimizer=keras.optimizers.Adam(),
              metrics=[mse_err],
              loss='mean_squared_error')
        loss=ratings_cross_validate(model,users,movies,ratings,
                               folds)
        results.append((F,loss))
        if loss<best_loss:
            best_loss=loss
            best_F=F
        print()
        print(f"==> {F},{best_penalty},{loss} == best ({best_F},{best_penalty},{best_loss}) =============")


F 5 :

============= Fold 1 ===========


/home/manel/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


	0: TRAIN Loss 1.5461,  Err 1.3691 || VAL Loss 1.4544, Err 1.2913
	1: TRAIN Loss 1.3842,  Err 1.2325 || VAL Loss 1.3235, Err 1.1829
	2: TRAIN Loss 1.2642,  Err 1.1331 || VAL Loss 1.2250, Err 1.1031
	3: TRAIN Loss 1.1733,  Err 1.0594 || VAL Loss 1.1498, Err 1.0437
	4: TRAIN Loss 1.1034,  Err 1.0041 || VAL Loss 1.0917, Err 0.9990
	5: TRAIN Loss 1.0489,  Err 0.9621 || VAL Loss 1.0461, Err 0.9650
	6: TRAIN Loss 1.0060,  Err 0.9298 || VAL Loss 1.0101, Err 0.9387
	7: TRAIN Loss 0.9718,  Err 0.9047 || VAL Loss 0.9813, Err 0.9184
	8: TRAIN Loss 0.9442,  Err 0.8850 || VAL Loss 0.9580, Err 0.9023
	9: TRAIN Loss 0.9218,  Err 0.8693 || VAL Loss 0.9390, Err 0.8896
	10: TRAIN Loss 0.9034,  Err 0.8567 || VAL Loss 0.9234, Err 0.8794
	11: TRAIN Loss 0.8881,  Err 0.8464 || VAL Loss 0.9105, Err 0.8711
	12: TRAIN Loss 0.8754,  Err 0.8381 || VAL Loss 0.8997, Err 0.8644
	13: TRAIN Loss 0.8647,  Err 0.8311 || VAL Loss 0.8907, Err 0.8589
	14: TRAIN Loss 0.8557,  Err 0.8253 || VAL Loss 0.8832, Err 0.8543
	15: 

	22: TRAIN Loss 0.7335,  Err 0.6790 || VAL Loss 0.7920, Err 0.7375
	23: TRAIN Loss 0.7334,  Err 0.6788 || VAL Loss 0.7929, Err 0.7382
	24: TRAIN Loss 0.7332,  Err 0.6785 || VAL Loss 0.7937, Err 0.7390
	25: TRAIN Loss 0.7331,  Err 0.6783 || VAL Loss 0.7945, Err 0.7396
	26: TRAIN Loss 0.7330,  Err 0.6781 || VAL Loss 0.7952, Err 0.7403
	27: TRAIN Loss 0.7329,  Err 0.6779 || VAL Loss 0.7959, Err 0.7408
	28: TRAIN Loss 0.7328,  Err 0.6777 || VAL Loss 0.7965, Err 0.7413
	29: TRAIN Loss 0.7327,  Err 0.6775 || VAL Loss 0.7971, Err 0.7418
	30: TRAIN Loss 0.7326,  Err 0.6773 || VAL Loss 0.7976, Err 0.7423
	31: TRAIN Loss 0.7325,  Err 0.6771 || VAL Loss 0.7982, Err 0.7427
	32: TRAIN Loss 0.7325,  Err 0.6770 || VAL Loss 0.7987, Err 0.7432
	33: TRAIN Loss 0.7324,  Err 0.6768 || VAL Loss 0.7991, Err 0.7435
	34: TRAIN Loss 0.7324,  Err 0.6767 || VAL Loss 0.7996, Err 0.7439
	35: TRAIN Loss 0.7323,  Err 0.6766 || VAL Loss 0.8000, Err 0.7443
	36: TRAIN Loss 0.7322,  Err 0.6764 || VAL Loss 0.8004, Err 0.

	44: TRAIN Loss 0.7312,  Err 0.6738 || VAL Loss 0.8006, Err 0.7432
	45: TRAIN Loss 0.7311,  Err 0.6738 || VAL Loss 0.8008, Err 0.7435
	46: TRAIN Loss 0.7311,  Err 0.6737 || VAL Loss 0.8011, Err 0.7437
	47: TRAIN Loss 0.7311,  Err 0.6737 || VAL Loss 0.8014, Err 0.7440
	48: TRAIN Loss 0.7311,  Err 0.6737 || VAL Loss 0.8015, Err 0.7441
	49: TRAIN Loss 0.7311,  Err 0.6736 || VAL Loss 0.8018, Err 0.7443
	50: TRAIN Loss 0.7310,  Err 0.6736 || VAL Loss 0.8020, Err 0.7445
	51: TRAIN Loss 0.7310,  Err 0.6735 || VAL Loss 0.8021, Err 0.7447
	52: TRAIN Loss 0.7310,  Err 0.6735 || VAL Loss 0.8023, Err 0.7448
	53: TRAIN Loss 0.7310,  Err 0.6735 || VAL Loss 0.8025, Err 0.7450
	54: TRAIN Loss 0.7310,  Err 0.6735 || VAL Loss 0.8026, Err 0.7451
	55: TRAIN Loss 0.7309,  Err 0.6734 || VAL Loss 0.8027, Err 0.7452
	56: TRAIN Loss 0.7309,  Err 0.6734 || VAL Loss 0.8029, Err 0.7454
	57: TRAIN Loss 0.7309,  Err 0.6733 || VAL Loss 0.8031, Err 0.7455
	58: TRAIN Loss 0.7309,  Err 0.6733 || VAL Loss 0.8032, Err 0.

...

	73: TRAIN Loss 0.7077,  Err 0.6202 || VAL Loss 0.8119, Err 0.7244
	74: TRAIN Loss 0.7077,  Err 0.6202 || VAL Loss 0.8120, Err 0.7245
	75: TRAIN Loss 0.7077,  Err 0.6201 || VAL Loss 0.8122, Err 0.7246
	76: TRAIN Loss 0.7077,  Err 0.6201 || VAL Loss 0.8123, Err 0.7248
	77: TRAIN Loss 0.7077,  Err 0.6201 || VAL Loss 0.8123, Err 0.7248
	78: TRAIN Loss 0.7077,  Err 0.6201 || VAL Loss 0.8125, Err 0.7249
	79: TRAIN Loss 0.7077,  Err 0.6201 || VAL Loss 0.8126, Err 0.7251
	80: TRAIN Loss 0.7076,  Err 0.6200 || VAL Loss 0.8128, Err 0.7252
	81: TRAIN Loss 0.7076,  Err 0.6200 || VAL Loss 0.8129, Err 0.7253
	82: TRAIN Loss 0.7076,  Err 0.6200 || VAL Loss 0.8131, Err 0.7254
	83: TRAIN Loss 0.7076,  Err 0.6200 || VAL Loss 0.8132, Err 0.7255
	84: TRAIN Loss 0.7076,  Err 0.6200 || VAL Loss 0.8133, Err 0.7257
	85: TRAIN Loss 0.7076,  Err 0.6199 || VAL Loss 0.8135, Err 0.7258
	86: TRAIN Loss 0.7076,  Err 0.6199 || VAL Loss 0.8135, Err 0.7259
	87: TRAIN Loss 0.7076,  Err 0.6199 || VAL Loss 0.8136, Err 0.

	93: TRAIN Loss 0.6995,  Err 0.5987 || VAL Loss 0.8482, Err 0.7474
	94: TRAIN Loss 0.6993,  Err 0.5984 || VAL Loss 0.8482, Err 0.7472
	95: TRAIN Loss 0.6992,  Err 0.5981 || VAL Loss 0.8482, Err 0.7471
	96: TRAIN Loss 0.6990,  Err 0.5978 || VAL Loss 0.8483, Err 0.7470
	97: TRAIN Loss 0.6989,  Err 0.5975 || VAL Loss 0.8483, Err 0.7469
	98: TRAIN Loss 0.6988,  Err 0.5972 || VAL Loss 0.8484, Err 0.7468
	99: TRAIN Loss 0.6987,  Err 0.5970 || VAL Loss 0.8484, Err 0.7466
======= fold 1 loss 0.7466241203771755 ============


============= Fold 2 ===========
	0: TRAIN Loss 0.7339,  Err 0.6331 || VAL Loss 0.7046, Err 0.6049
	1: TRAIN Loss 0.7270,  Err 0.6280 || VAL Loss 0.7140, Err 0.6157
	2: TRAIN Loss 0.7220,  Err 0.6241 || VAL Loss 0.7227, Err 0.6253
	3: TRAIN Loss 0.7182,  Err 0.6211 || VAL Loss 0.7307, Err 0.6338
	4: TRAIN Loss 0.7153,  Err 0.6186 || VAL Loss 0.7379, Err 0.6414
	5: TRAIN Loss 0.7129,  Err 0.6165 || VAL Loss 0.7446, Err 0.6483
	6: TRAIN Loss 0.7110,  Err 0.6146 || VAL Loss 0

	13: TRAIN Loss 0.6992,  Err 0.5971 || VAL Loss 0.7750, Err 0.6728
	14: TRAIN Loss 0.6987,  Err 0.5963 || VAL Loss 0.7784, Err 0.6760
	15: TRAIN Loss 0.6982,  Err 0.5957 || VAL Loss 0.7814, Err 0.6789
	16: TRAIN Loss 0.6978,  Err 0.5951 || VAL Loss 0.7843, Err 0.6816
	17: TRAIN Loss 0.6974,  Err 0.5945 || VAL Loss 0.7871, Err 0.6841
	18: TRAIN Loss 0.6970,  Err 0.5939 || VAL Loss 0.7895, Err 0.6864
	19: TRAIN Loss 0.6967,  Err 0.5935 || VAL Loss 0.7918, Err 0.6885
	20: TRAIN Loss 0.6964,  Err 0.5930 || VAL Loss 0.7941, Err 0.6906
	21: TRAIN Loss 0.6962,  Err 0.5926 || VAL Loss 0.7961, Err 0.6924
	22: TRAIN Loss 0.6960,  Err 0.5921 || VAL Loss 0.7981, Err 0.6942
	23: TRAIN Loss 0.6957,  Err 0.5918 || VAL Loss 0.7999, Err 0.6959
	24: TRAIN Loss 0.6955,  Err 0.5914 || VAL Loss 0.8017, Err 0.6975
	25: TRAIN Loss 0.6953,  Err 0.5911 || VAL Loss 0.8033, Err 0.6990
	26: TRAIN Loss 0.6952,  Err 0.5908 || VAL Loss 0.8048, Err 0.7004
	27: TRAIN Loss 0.6950,  Err 0.5905 || VAL Loss 0.8063, Err 0.

	35: TRAIN Loss 0.6933,  Err 0.5867 || VAL Loss 0.8133, Err 0.7066
	36: TRAIN Loss 0.6933,  Err 0.5865 || VAL Loss 0.8142, Err 0.7075
	37: TRAIN Loss 0.6932,  Err 0.5863 || VAL Loss 0.8151, Err 0.7083
	38: TRAIN Loss 0.6931,  Err 0.5862 || VAL Loss 0.8159, Err 0.7089
	39: TRAIN Loss 0.6930,  Err 0.5860 || VAL Loss 0.8166, Err 0.7096
	40: TRAIN Loss 0.6930,  Err 0.5859 || VAL Loss 0.8174, Err 0.7103
	41: TRAIN Loss 0.6929,  Err 0.5857 || VAL Loss 0.8181, Err 0.7109
	42: TRAIN Loss 0.6928,  Err 0.5856 || VAL Loss 0.8188, Err 0.7116
	43: TRAIN Loss 0.6928,  Err 0.5855 || VAL Loss 0.8195, Err 0.7121
	44: TRAIN Loss 0.6927,  Err 0.5853 || VAL Loss 0.8201, Err 0.7127
	45: TRAIN Loss 0.6927,  Err 0.5852 || VAL Loss 0.8207, Err 0.7132
	46: TRAIN Loss 0.6926,  Err 0.5851 || VAL Loss 0.8213, Err 0.7138
	47: TRAIN Loss 0.6926,  Err 0.5849 || VAL Loss 0.8219, Err 0.7143
	48: TRAIN Loss 0.6926,  Err 0.5849 || VAL Loss 0.8225, Err 0.7148
	49: TRAIN Loss 0.6925,  Err 0.5848 || VAL Loss 0.8230, Err 0.

	57: TRAIN Loss 0.6929,  Err 0.5841 || VAL Loss 0.8226, Err 0.7138
	58: TRAIN Loss 0.6929,  Err 0.5840 || VAL Loss 0.8229, Err 0.7140
	59: TRAIN Loss 0.6928,  Err 0.5839 || VAL Loss 0.8233, Err 0.7144
	60: TRAIN Loss 0.6928,  Err 0.5839 || VAL Loss 0.8236, Err 0.7147
	61: TRAIN Loss 0.6928,  Err 0.5838 || VAL Loss 0.8239, Err 0.7150
	62: TRAIN Loss 0.6928,  Err 0.5838 || VAL Loss 0.8243, Err 0.7153
	63: TRAIN Loss 0.6928,  Err 0.5837 || VAL Loss 0.8246, Err 0.7155
	64: TRAIN Loss 0.6927,  Err 0.5836 || VAL Loss 0.8250, Err 0.7160
	65: TRAIN Loss 0.6927,  Err 0.5836 || VAL Loss 0.8253, Err 0.7161
	66: TRAIN Loss 0.6927,  Err 0.5835 || VAL Loss 0.8256, Err 0.7164
	67: TRAIN Loss 0.6927,  Err 0.5835 || VAL Loss 0.8258, Err 0.7167
	68: TRAIN Loss 0.6927,  Err 0.5834 || VAL Loss 0.8262, Err 0.7169
	69: TRAIN Loss 0.6926,  Err 0.5834 || VAL Loss 0.8263, Err 0.7171
	70: TRAIN Loss 0.6926,  Err 0.5834 || VAL Loss 0.8266, Err 0.7173
	71: TRAIN Loss 0.6926,  Err 0.5834 || VAL Loss 0.8268, Err 0.

Best model has $F=15$ and $\gamma=2e-5$.

## Test of Best Model

In [ ]:
              
mu=ratings.mean()
dratings=ratings-mu
dratings_test=ratings_test-mu

In [111]:
model=collaborative_filter(N_users,N_movies,best_F,best_penalty)
model.compile(optimizer=keras.optimizers.Adam(),
              metrics=[mse_err],
              loss='mean_squared_error')


In [113]:
model.fit([users,movies],dratings,
          epochs=200,batch_size=5000,
          callbacks=[ReportCallback(1,True)],
          verbose=0,
          validation_data=([users_test,movies_test],dratings_test))

	0: TRAIN Loss 0.7223,  Err 0.6430 || VAL Loss 0.8160, Err 0.7364
	1: TRAIN Loss 0.7215,  Err 0.6414 || VAL Loss 0.8163, Err 0.7358
	2: TRAIN Loss 0.7209,  Err 0.6399 || VAL Loss 0.8165, Err 0.7352
	3: TRAIN Loss 0.7202,  Err 0.6385 || VAL Loss 0.8167, Err 0.7347
	4: TRAIN Loss 0.7197,  Err 0.6372 || VAL Loss 0.8170, Err 0.7342
	5: TRAIN Loss 0.7191,  Err 0.6359 || VAL Loss 0.8172, Err 0.7337
	6: TRAIN Loss 0.7186,  Err 0.6347 || VAL Loss 0.8175, Err 0.7333
	7: TRAIN Loss 0.7181,  Err 0.6336 || VAL Loss 0.8177, Err 0.7329
	8: TRAIN Loss 0.7177,  Err 0.6326 || VAL Loss 0.8179, Err 0.7326
	9: TRAIN Loss 0.7172,  Err 0.6316 || VAL Loss 0.8181, Err 0.7322
	10: TRAIN Loss 0.7169,  Err 0.6307 || VAL Loss 0.8183, Err 0.7319
	11: TRAIN Loss 0.7165,  Err 0.6298 || VAL Loss 0.8184, Err 0.7316
	12: TRAIN Loss 0.7161,  Err 0.6289 || VAL Loss 0.8187, Err 0.7313
	13: TRAIN Loss 0.7158,  Err 0.6281 || VAL Loss 0.8189, Err 0.7310
	14: TRAIN Loss 0.7155,  Err 0.6273 || VAL Loss 0.8190, Err 0.7307
	15: 

	123: TRAIN Loss 0.7068,  Err 0.6073 || VAL Loss 0.8193, Err 0.7199
	124: TRAIN Loss 0.7068,  Err 0.6072 || VAL Loss 0.8193, Err 0.7198
	125: TRAIN Loss 0.7068,  Err 0.6072 || VAL Loss 0.8193, Err 0.7198
	126: TRAIN Loss 0.7068,  Err 0.6072 || VAL Loss 0.8193, Err 0.7198
	127: TRAIN Loss 0.7068,  Err 0.6071 || VAL Loss 0.8193, Err 0.7197
	128: TRAIN Loss 0.7067,  Err 0.6071 || VAL Loss 0.8193, Err 0.7197
	129: TRAIN Loss 0.7067,  Err 0.6070 || VAL Loss 0.8193, Err 0.7197
	130: TRAIN Loss 0.7067,  Err 0.6070 || VAL Loss 0.8192, Err 0.7196
	131: TRAIN Loss 0.7067,  Err 0.6070 || VAL Loss 0.8193, Err 0.7196
	132: TRAIN Loss 0.7067,  Err 0.6070 || VAL Loss 0.8192, Err 0.7196
	133: TRAIN Loss 0.7066,  Err 0.6069 || VAL Loss 0.8192, Err 0.7196
	134: TRAIN Loss 0.7067,  Err 0.6069 || VAL Loss 0.8193, Err 0.7196
	135: TRAIN Loss 0.7066,  Err 0.6069 || VAL Loss 0.8193, Err 0.7196
	136: TRAIN Loss 0.7066,  Err 0.6069 || VAL Loss 0.8193, Err 0.7195
	137: TRAIN Loss 0.7066,  Err 0.6068 || VAL Loss

In [114]:
Y_pred=mu+model.predict([users_test,movies_test]).ravel()
loss=np.mean((ratings_test-Y_pred)**2)
print(best_F,best_penalty,loss)

15 2e-05 0.7187691488387822


Result is comparable to what we obtained with `numpy` (0.726 versus 0.718)